In [1]:
import astroRMS

In [2]:
def write_rms_map(i2d_file, input_type='wht'):
    # Load sci/wht data
    sci_hd = fits.getheader(i2d_file, 'sci')
    sci_data = fits.getdata(i2d_file, 'sci')
    wht_hd = fits.getheader(i2d_file, 'err')
    # Choose the weight or error data
    if input_type=='wht':
        wht_data = fits.getdata(i2d_file, 'wht')
    elif input_type=='err':
        wht_data = 1 / fits.getdata(i2d_file, 'err')**2
    else:
        ValueError('input_type not known')
    # Write temporary sci/wht files
    sci_file = 'temp_sci.fits'
    wht_file = 'temp_wht.fits'
    fits.writeto(sci_file, sci_data, header=sci_hd, overwrite=True)
    fits.writeto(wht_file, wht_data, header=wht_hd, overwrite=True)
    # Run astroRMS
    out_file = i2d_file.replace('i2d', 'rms')
    astroRMS.create_error_map(sci_file, wht_file, out_file, map_type='rms')
    # Clean up the temporary file
    os.system('rm temp_sci.fits temp_wht.fits')
    # Calculate the RMS scaling factor 
    rms_data = fits.getdata(out_file)
    use_idxs = wht_data>0
    err_scl_fac = (rms_data[use_idxs] / (1/wht_data[use_idxs])**0.5)[0]
    return err_scl_fac
